In [1]:
import pyconll #pip3 install this if you don't have it
import torchtext.data as tt
import torch 
import torch.nn as nn
import torch.optim as optim

In [2]:
AFRIKAANS_TRAIN = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-train.conllu'
AFRIKAANS_DEV = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-dev.conllu'
AFRIKAANS_TEST = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-test.conllu'

In [3]:
# from https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb
def make_sentences(path):
    data = pyconll.load_from_file(path)
    sentences = []
    tagged_sentences = []
    for each in data:
        tagged_sentence=[]
        sentence = []
        for token in each:
            if token.upos and token.form:
                tagged_sentence.append(token.upos)
                sentence.append(token.form.lower())
        tagged_sentences.append(tagged_sentence)
        sentences.append(sentence)
    return sentences, tagged_sentences

In [9]:
train_afr, tagged_train_afr = make_sentences(AFRIKAANS_TRAIN)
dev_afr, tagged_dev_afr = make_sentences(AFRIKAANS_DEV)
test_afr, tagged_test_afr = make_sentences(AFRIKAANS_TEST)

In [10]:
print(tagged_train_afr[0])

['DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'AUX', 'ADP', 'DET', 'NUM', 'NOUN', 'ADP', 'NOUN', 'PART', 'VERB', 'ADP', 'NOUN', 'PRON', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'AUX', 'PUNCT']


In [11]:
print("Tagged sentences in train set: ", len(tagged_train_afr))
print("Tagged words in train set:", len([item for sublist in tagged_train_afr for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(tagged_dev_afr))
print("Tagged words in dev set:", len([item for sublist in tagged_dev_afr for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(tagged_test_afr))
print("Tagged words in test set:", len([item for sublist in tagged_test_afr for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(tagged_train_afr)+len(tagged_dev_afr)+len(tagged_dev_afr))

Tagged sentences in train set:  1315
Tagged words in train set: 33894
Tagged sentences in dev set:  194
Tagged words in dev set: 5317
Tagged sentences in test set:  425
Tagged words in test set: 10065
****************************************
Total sentences in dataset: 1703


In [12]:
# from https://github.com/tringm/POSTagger_Pytorch/blob/master/src/util/nlp.py
def build_tag_field(sentences_tokens):
    token_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('tokens', token_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_tokens]
    torch_dataset = tt.Dataset(examples, fields)
    token_field.build_vocab(torch_dataset)
    return token_field
    
def build_text_field(sentences_words):
    text_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('text', text_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_words]
    torch_dataset = tt.Dataset(examples, fields)
    text_field.build_vocab(torch_dataset)
    return text_field

In [13]:
train_afr = build_text_field(train_afr)
dev_afr = build_text_field(dev_afr)
test_afr = build_text_field(test_afr)

tagged_train_afr = build_tag_field(tagged_train_afr)
tagged_dev_afr = build_tag_field(tagged_dev_afr)
tagged_test_afr = build_tag_field(tagged_test_afr)

train_data = tt.Dataset([],(("text", train_afr), ("udtags", tagged_train_afr)))
valid_data = tt.Dataset([],(("text", dev_afr), ("udtags", tagged_dev_afr)))
test_data = tt.Dataset([],(("text", test_afr), ("udtags", tagged_test_afr)))

In [18]:
print(train_afr.vocab)

In [14]:
print(train_data)

In [22]:
# from https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
#model
batch_size=128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#needs to be tuple of dataset objects
train_iterator, valid_iterator, test_iterator = tt.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [45]:
class BiLSTMTagger(nn.Module):
    #https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout = dropout if n_layers > 1 else 0)
        #fully connected layer
        self.fc = nn.Linear((hidden_dim * 2 if bidirectional else hidden_dim), output_dim)
        self.dropout = nn.Dropout(dropout)
     
    
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        return predictions

In [46]:
in_dim = len(train_afr.vocab)
emb_dim = 100
hid_dim = 128
out_dim = len(tagged_train_afr.vocab)
n_layers = 2
bidirectional = True
dropout = 0.25
pad_index = train_afr.vocab.stoi[train_afr.pad_token]

In [49]:
model = BiLSTMTagger(in_dim, emb_dim, hid_dim, out_dim, n_layers, bidirectional, dropout, pad_index)

In [48]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMTagger(
  (embedding): Embedding(4677, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=256, out_features=20, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [37]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

AttributeError: 'BiLSTMTagger' object has no attribute 'parameters'